In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import os
import cv2

In [3]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
data_path = "Waste_Classification_data"

In [5]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    return img

images = []
labels = []


label_map = {'Recyclable': 0, 'Compostable': 1, 'Harzardous': 2}


for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = load_image(file_path)
        images.append(img)
        labels.append(label_map[folder])


images = np.array(images)
labels = np.array(labels)

In [6]:
from keras.utils import to_categorical

labels = np.array(labels)
labels = to_categorical(labels, num_classes=3)

In [7]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=19)

In [8]:
from keras.models import Sequential
from keras.layers import Dense

In [9]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation='softmax')
])

C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
history = model.fit(train_images, train_labels, epochs=EPOCHS, validation_data=(test_images, test_labels))

Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 73s 934ms/step - accuracy: 0.5929 - loss: 1.2563 - val_accuracy: 0.8534 - val_loss: 0.3884
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 55s 748ms/step - accuracy: 0.8743 - loss: 0.3077 - val_accuracy: 0.8707 - val_loss: 0.3048
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 52s 718ms/step - accuracy: 0.8746 - loss: 0.3033 - val_accuracy: 0.9310 - val_loss: 0.1862
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 53s 720ms/step - accuracy: 0.9192 - loss: 0.2092 - val_accuracy: 0.8966 - val_loss: 0.2694
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 55s 746ms/step - accuracy: 0.9175 - loss: 0.2221 - val_accuracy: 0.9431 - val_loss: 0.1437
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 53s 730ms/step - accuracy: 0.9396 - loss: 0.1654 - val_accuracy: 0.9397 - val_loss: 0.1727
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 57s 778ms/step - accuracy: 0.9438 - loss: 0.1576 - val_accuracy: 0.9414 - val_loss: 0.1674
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 55s 749ms/step - accuracy: 0.9422 - loss: 0.1515 - val_accu

In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc:.2f}')

19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 159ms/step - accuracy: 0.9387 - loss: 0.2231
Test accuracy: 0.93


In [24]:
predictions = model.predict(test_images)

19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step


In [ ]:
for i, prediction in enumerate(predictions):
    img = test_images[i]
    img = (img * 255).astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    label = np.argmax(prediction)
    if label == 0:
        cv2.putText(img, 'Recyclable', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    elif label == 1:
        cv2.putText(img, 'Compostable', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        cv2.putText(img, 'Hazardous', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imshow('Prediction', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [26]:
model.save('model.h5')

In [38]:
import zipfile

with zipfile.ZipFile('model.h5.zip', 'w') as zip_file:
    zip_file.write('model.h5')